In [2]:
import dataloader
import torch
import dgl
import numpy as np

# Load graph ===================================
graph = dataloader.load_DGL('cora')
feat = graph.ndata['feat']
labels = graph.ndata['label']
adj = graph.adj().to_dense()

  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.


In [37]:
def save_as_dgl(graph, adj, g0, name, root='./locked/'):
  edges = adj.to_sparse().indices()
  d = dgl.graph((edges[0], edges[1]), num_nodes=graph.num_nodes())
  d.ndata['g0'] = g0
  dgl.data.utils.save_graphs(f'{root}{name}.bin', [d], {"glabel": torch.tensor([0])})

def load_dgl(name, root='./locked/') -> dgl.DGLGraph:
  d = dgl.load_graphs(f'{root}{name}.bin')[0][0]
  return d

save_as_dgl(graph, adj, g0, 'test')
load_dgl('test')

Graph(num_nodes=2708, num_edges=10556,
      ndata_schemes={'g0': Scheme(shape=(), dtype=torch.uint8)}
      edata_schemes={})

In [14]:
g0_probs = torch.ones(graph.num_nodes())
g0_probs = g0_probs * (270 / g0_probs.sum())
g0_probs.clamp_(0, 1)
g0 = torch.bernoulli(g0_probs).bool()

In [20]:
labels[g0].shape, adj[g0, :][:, g0].shape

(torch.Size([243]), torch.Size([243, 243]))

In [24]:
def calc_homophily(adj: torch.tensor, labels: torch.tensor, mask: torch.tensor=None) -> float:
  """
  returns H (number of similar edge / number of edges)
  """
  if mask != None:
    adj = adj[mask, :][:, mask]
    labels = labels[mask]
    
  edges = adj.nonzero().t()
  match = labels[edges[0]] == labels[edges[1]]

  return match.sum().item() / match.shape[0]

calc_homophily(adj, labels)

0.8099658961727927